***Задание 1***

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [28]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup


In [36]:
KEYWORDS = ['sql', 'bigdata', 'data']
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')

In [38]:
news_list = []
for post in posts:
    post_id = post.parent.attrs.get('id')
   
    if not post_id:
        continue
    
    post_id = int(post_id.split('_')[-1])
    text_preview = post.find_all('div', class_='post__text')
    
    for preview in text_preview:
        preview_lower = preview.text.lower()
        if any([desired in preview_lower for desired in KEYWORDS]):
                
            date_element = post.find('span', class_ = 'post__time')
            title_element = post.find('a', class_='post__title_link')
            link_element = title_element.attrs.get('href')
           
            news_list.append({'Дата': date_element.text, 'Заголовок': title_element.text, 'Ссылка': link_element})
            
df = pd.DataFrame(news_list)
df

,Дата,Заголовок,Ссылка
0,вчера в 18:12,Введение в Spring Data JDBC,https://habr.com/ru/company/otus/blog/531332/
1,вчера в 17:41,Практические методы оптимизации запросов в Apa...,https://habr.com/ru/post/518886/


***Задание 2***

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [ ]:
import json
import datetime

In [14]:
time.sleep(0.3)

url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
params = {
    "emailAddresses":['xxxx@gmail.ru', 'yyyy@rambler.ru', 'elmelkumowa@gmail.com', 'el.melkumowa@yandex.ru'],
}
headers = {
    'Vaar-Version':'0',
    'Vaar-Header-App-Product':'hackcheck-web-avast'
}

res = requests.post(url, headers=headers, data=json.dumps(params))
res.json()

leak_email = {}
data = pd.DataFrame()

for email, leak in res.json()['summary'].items():
    leak_email[email] = list(leak.values())[0]
for mail_id, details in res.json()['breaches'].items():
    for email, leak in leak_email.items():
        if int(mail_id) in leak:
            row = {'Почта': email, 
                   'Дата утечки': details['publishDate'], 
                   'Источник утечки': details['site'],
                   'Описание утечки': details['description']
                  }
            data = pd.concat([data, pd.DataFrame([row])])

data

,Почта,Дата утечки,Источник утечки,Описание утечки
0,xxxx@gmail.ru,2019-03-07T00:00:00Z,dubsmash.com,"In December 2018, Dubsmash's database was alle..."
0,el.melkumowa@yandex.ru,2017-06-24T00:00:00Z,gamenet.ru,"At an unconfirmed date, GameNet's user databas..."
0,yyyy@rambler.ru,2016-10-21T00:00:00Z,myspace.com,"Shortly before the 2016 Memorial Day weekend, ..."
0,yyyy@rambler.ru,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro..."
0,xxxx@gmail.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,yyyy@rambler.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,yyyy@rambler.ru,2017-03-15T00:00:00Z,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
0,yyyy@rambler.ru,2016-10-23T00:00:00Z,rambler.ru,"On February 17, 2012, Russian email provider R..."
0,yyyy@rambler.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,yyyy@rambler.ru,2017-03-31T00:00:00Z,fc-zenit.ru,"In July 2010, FC Zenit's user database was all..."
